# Lista 1

In [23]:
import numpy as np

from gurobipy import Model, GRB

## Exercício 1

In [4]:
# Create a new model
m1 = Model('Máquinas')

# Create variables
x1 = m1.addVar(lb=0, vtype=GRB.INTEGER, name='x1')
x2 = m1.addVar(lb=0, vtype=GRB.INTEGER, name='x2')

# Set objective
m1.setObjective(4*x1 + 2.5*x2, GRB.MAXIMIZE)

# Add constraints
m1.addConstr(0.75*x1 + 0.5*x2 <= 8, 'M1')
m1.addConstr(0.75*x1 + 0.75*x2 <= 7, 'M2')

m1.update()

%time m1.optimize()

for v in m1.getVars():
    print('%s %g' % (v.varName, v.x))

print('Obj: %g' % m1.objVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xbb97aa4e
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e-01, 8e-01]
  Objective range  [2e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+00, 8e+00]
Found heuristic solution: objective 36.0000000
Presolve removed 2 rows and 2 columns
Presolve time: 0.08s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.13 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 36 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.600000000000e+01, best bound 3.600000000000e+01, gap 0.0000%
CPU times: user 20.5 ms, sys: 4.08 ms, total: 24.6 ms
Wall time: 154 ms
x1 9
x2 -0
Obj: 36


In [13]:
p1 = x1.x / 4
p2 = x2.x / 4

print(f'Solution: {p1} p1 and {p2} p2')

Solution: 2.25 p1 and -0.0 p2


## Exercício 2

In [53]:
# Create a new model
m2 = Model('Sucatas')

# Create variables
q = m2.addMVar(2, lb=[0, 0], ub=[1, 1],
               vtype=GRB.CONTINUOUS, name='q')

# Set objective
c = np.array([100, 80])
m2.setObjective(c @ q, GRB.MINIMIZE)

# Add constraints
A = np.array([
    [0.06, 0.03],
    [0.03, 0.06],
    [0.04, 0.03],
])
u = np.array([0.06, 0.05, 0.07])
l = np.array([0.03, 0.03, 0.03])

m2.addConstr(A @ q >= l, 'C_l')
m2.addConstr(A @ q <= u, 'C_u')
m2.addConstr(q[0] + q[1] == 1, 'C_q')

m2.update()

%time m2.optimize()

for v in m2.getVars():
    print('%s %g' % (v.varName, v.x))

print('Obj: %g' % m2.objVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 7 rows, 2 columns and 14 nonzeros
Model fingerprint: 0x7da5535e
Coefficient statistics:
  Matrix range     [3e-02, 1e+00]
  Objective range  [8e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-02, 1e+00]
Presolve removed 7 rows and 2 columns
Presolve time: 0.22s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.6666667e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.22 seconds
Optimal objective  8.666666667e+01
CPU times: user 14.4 ms, sys: 3.83 ms, total: 18.3 ms
Wall time: 236 ms
q[0] 0.333333
q[1] 0.666667
Obj: 86.6667


## Exercício 3

In [72]:
# Create a new model
m3 = Model('Depósito')

# Create variables
clients = [
    (np.array([5, 10]), 200),
    (np.array([10, 5]), 150),
    (np.array([0, 12]), 200),
    (np.array([12, 0]), 300),
]

p = m3.addMVar(2, vtype=GRB.CONTINUOUS, name='pos')
# auxiliary variable for calculations
z = m3.addMVar((len(clients), 2), vtype=GRB.CONTINUOUS, name='aux')

s = list()
for (q_i, n_i), z_i in zip(clients, z):
    m3.addConstr(z_i == p - q_i)
    
    # we omit the sqrt as optimizing the inner product
    # also optimizes its sqrt
    s.append((z_i @ z_i) * n_i)

# Set objective
m3.setObjective(sum(s), GRB.MINIMIZE)

m3.update()

%time m3.optimize()

for v in m3.getVars():
    print('%s %g' % (v.varName, v.x))

print('Obj: %g' % m3.objVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 8 rows, 10 columns and 16 nonzeros
Model fingerprint: 0x72752115
Model has 8 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [3e+02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 1e+01]
Presolve removed 2 rows and 2 columns
Presolve time: 0.12s
Presolved: 6 rows, 8 columns, 12 nonzeros
Presolved model has 8 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 6.000e+00
 Factor NZ  : 1.200e+01
 Factor Ops : 2.800e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.71555000e+06 -1.71555000e+06  0.00e+00 0.00e+00  9.13e+05     0s
   1   5.60069931e+05 -4.07638929